In [8]:
import pandas as pd
import numpy as np

L_classes = [["2nde",17,4.5],["2nde_ND",1,3],["SL",3,1.5],["1ES",12,1.5],["1SPC",8,4],["TES",12,1.5],["TSPC",4,6],["TSI",2,2],["1STI",3,6],["TSTI",3,6]]
L_qu = ["Combien de 2nde ?","2 non dédoublée ?","Combien de SL ?","Combien de 1ES ?","Combien de 1SPC ?","Combien de TES ?","Combien de TSPC ?","Combien de T-SI ?","Combien de 1STI ?","Combien de TSTI ?"]

nb, noms, classes = [],[],[]

for i in range(len(L_qu)):
    nb.append(0)

def init() :
    tableau = pd.DataFrame({'Noms' : [],'2nde' : [],'2nde_ND' : [],'SL' : [],'1ES' : [],'1SPC' : [],'TES' : [],'TSPC' : [],'TSI' : [],\
                        '1STI' : [],'TSTI' : [],'Service_dû' : [],'Service_effectif' : [],'Diff' : []},\
                       columns=['Noms','2nde','2nde_ND','SL','1ES','1SPC','TES','TSPC','TSI','1STI','TSTI','Service_dû','Service_effectif','Diff'])
    return tableau

tableau = init()
tableau.set_index('Noms', inplace=True)
    
def voeux(tableau) :
    row = []
    nom = input("Nom ?")
    service_du = int(input("Service ?"))
    service_eff = 0
    row += [nom]
    for i in range(len(L_classes)) :
        nb[i] = int(input(L_qu[i]))
        L_classes[i][1] -= nb[i]
        row += [nb[i]]
        service_eff += nb[i]*L_classes[i][2]
    row += [service_du]
    row += [service_eff]
    row += [service_eff-service_du]
    tableau.loc[len(tableau)] = row
    return tableau

def raz(tableau) :
    for i in range(1,len(tableau)) :
        tableau[i]=[]
        
def lire() :
    df = pd.read_csv('recap.csv',sep=',')
    df.apply(pd.to_numeric, errors='ignore')
    return df

def problemes(tableau) :
    print()
    s = []
    pofs_oui = []
    profs_non = []
    b = pd.DataFrame(L_classes)
    b = b[1]
    c = tableau.sum(axis = 0, skipna = True)[1:-2]
    c = pd.to_numeric(c)
    for i in range(len(L_classes)) :
        if c[i]<b[i] :
            print("- Il manque {} {}".format(int(b[i]-c[i]),L_classes[i][0]))
            sans = tableau[L_classes[i][0]] == 0
            profs_sans = tableau[sans]['Noms']
            print("profs n'en ayant pas :",*profs_sans,sep='  ',end="")
            print()
        if c[i]>b[i] :
            print("- Il y a {} {} en trop".format(int(c[i]-b[i]),L_classes[i][0]))
            avec = tableau[L_classes[i][0]] >= 1
            profs_avec = tableau[avec]['Noms']
            nb = tableau[avec][L_classes[i][0]]
            print("profs en ayant déjà : ",end="")
            for k in range(len(profs_avec)) :
                print("{} ({})".format(list(profs_avec)[k],int(list(nb)[k])),end=" ")
            print()

    print()
    print("Profs n'ayant pas leur quota :")
    endessous = tableau["Diff"] < 0
    profs_endessous = tableau[endessous]['Noms']
    nbh1 = tableau[endessous]['Diff']
    for k in range(len(profs_endessous)) :
        print("{} ({} h)".format(list(profs_endessous)[k],list(nbh1)[k]),end=" ")
    print("")
    print("\nProfs avec bcp d'heures :")
    audessus = tableau["Diff"] > 2
    profs_audessus = tableau[audessus]['Noms']
    nbh2 = tableau[audessus]['Diff']
    for k in range(len(profs_audessus)) :
        print("{} ({} h)".format(list(profs_audessus)[k],list(nbh2)[k]),end=" ")
        
def change(tableau) :
    nom = input("Nom ?")
    classe = input("Classe ?")
    var = input("Ajout = '1', Suppression = '-1'")
    tableau.loc[tableau['Noms'] == nom, classe] += int(var)
    return tableau
    
def sauve(tableau) :
    tableau.to_csv('recap.csv', index = False)

In [10]:
voeux(tableau)

Nom ?Marc
Service ?15
Combien de 2nde ?0
2 non dédoublée ?0
Combien de SL ?0
Combien de 1ES ?1
Combien de 1SPC ?0
Combien de TES ?1
Combien de TSPC ?0
Combien de T-SI ?0
Combien de 1STI ?1
Combien de TSTI ?1


ValueError: cannot set a row with mismatched columns

In [9]:
tableau=lire()

FileNotFoundError: [Errno 2] File recap.csv does not exist: 'recap.csv'

In [43]:
problemes(tableau)


- Il manque 9 2nde
profs n'en ayant pas :  David B  Marc  Jean-Pierre
- Il y a 1 2nde_ND en trop
profs en ayant déjà : Philippe (1) Catherine B (1) 
- Il manque 4 1ES
profs n'en ayant pas :  Jean-Pierre  Nicolas
- Il manque 2 1SPC
profs n'en ayant pas :  Marc  Julie  Doriane  Nicolas
- Il manque 8 TES
profs n'en ayant pas :  David B  Julie  Jean-Pierre  Nicolas
- Il manque 1 TSPC
profs n'en ayant pas :  Philippe  David B  Marc  Catherine B  Nicolas
- Il manque 1 TSI
profs n'en ayant pas :  David B  Marc  Julie  Jean-Pierre  Catherine B  Doriane  Nicolas
- Il manque 1 1STI
profs n'en ayant pas :  Philippe  Julie  Jean-Pierre  Catherine B  Doriane  Nicolas
- Il manque 2 TSTI
profs n'en ayant pas :  Philippe  David B  Julie  Jean-Pierre  Catherine B  Doriane  Nicolas

Profs n'ayant pas leur quota :
Nicolas (-9.0 h) 

Profs avec bcp d'heures :


In [70]:
change(tableau)

Nom ? Philippe
Classe ? SL
Ajout = '1', Suppression = '-1' 1


,Noms,2nde,2nde_ND,SL,1ES,1SPC,TES,TSPC,TSI,1STI,TSTI,Service_dû,Service_effectif,Diff
0,Philippe,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,18.0,18.0,0.0
1,David B,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,18.0,18.5,0.5
2,Marc,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,15.0,15.0,0.0
3,Julie,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,18.0,0.0
4,Jean-Pierre,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,9.0,10.0,1.0
5,Catherine B,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,18.0,18.5,0.5
6,Doriane,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,18.0,19.5,1.5
7,Nicolas,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,9.0,-9.0


In [ ]:
# tableau = tableau.drop([1]) #Enlève la ligne numéro i (pas sûr que besoin du tableau=)

In [64]:
tableau.loc[tableau['Noms'] == 'Marc','2nde']+=1
tableau

,Noms,2nde,2nde_ND,SL,1ES,1SPC,TES,TSPC,TSI,1STI,TSTI,Service_dû,Service_effectif,Diff
0,Philippe,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,18.0,18.0,0.0
1,David B,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,18.0,18.5,0.5
2,Marc,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,15.0,15.0,0.0
3,Julie,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,18.0,0.0
4,Jean-Pierre,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,9.0,10.0,1.0
5,Catherine B,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,18.0,18.5,0.5
6,Doriane,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,18.0,19.5,1.5
7,Nicolas,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,9.0,-9.0
